In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'planet-understanding-the-amazon-from-space:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F6322%2F868312%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240604%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240604T225305Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3e276d304f02a258311781b420a815300c35e51bd75256e54e3ac8240a72e0b9ddbec2d2e463fd8e66e1303f55c78cbb33a64afb57afdeeeb935ff60daaf7e2da7769c398fcb2a5d3aafdd8be009e4832fdf50fcc9babae6327d082ce370277d7707f5b7d342e55b80d65cd95347a8fe45c2fcb8e31230ff0441e7eaca03854cd4c07aff9c87f0b73c137d5de75f2937e33c2af96efceed7137b54d57e7af50c9fd67964479ee6b5341b1130db09a79ab6cf4525cce5eb02653de4824d10ac5dc975448fb66c336903b3ad136ef811987f016fb60fccadee186814d345647df7241741ef7d951efd52f39e70bbb7d63aa5d05b148534766ece2bb890bcac166f,planets-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F503255%2F938046%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240604%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240604T225305Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4ae5c53a9cf71d00aa2b2ca346075cf0dae863c84b3e5524668a5ac37671464a8651e033d43d266f58fd275c26931a0062d1dffd06955eda888791a832ce8b82d0e23d40ab0718753374f3ce8bbdc8df3b248dbe2d74a974f1719dd84249fa51db3f543b5f2da31ce5bce53173da2f3dc23645d1f0c5d5a5b16e26b4a2480265a208cd9082395a84e85f12c426b1b9cae67a912e2760bbe1186f28f8323ebd3412398db56196b9809a8e734c62562613ba1d1ae711a87fa9313dfff4096793f867078b08139649f1689d544d29dadf8815176129467faa014f3526e213c6e57f988c6a29a8028d18b2fd3081e1a1a50044500bfaa877e0dbf5f0294947bb978c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


## **Labeling of satellite images based on the different atmospheric conditions and different terrain regions using CNN.**

### **Importing the libraries**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
import gc
from keras import backend as K


## **Loading the datasets**

In [ ]:
#reading the labels
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

In [ ]:
encoder = LabelEncoder()

In [ ]:
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

In [ ]:
#defining a dict of encoded labels

label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [ ]:
#loading the traing_images

X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [ ]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

In [ ]:
X = np.array(X, np.float16)/255.0
Y = np.array(Y)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

In [ ]:
del X, Y
gc.collect()

In [ ]:
gc.collect()


In [ ]:
def fbeta(y_true, y_pred, beta=2, threshold=0.0):
    y_pred = K.clip(y_pred, min_value=0.0, max_value=1.0)
    y_pred_bin = K.round(y_pred + threshold)
    tp = K.sum(K.cast(y_true * y_pred_bin, dtype='float32')) + K.epsilon()
    fp = K.sum(K.cast(K.clip(y_pred_bin - y_true, 0, 1), dtype='float32'))
    fn = K.sum(K.cast(K.clip(y_true - y_pred, 0, 1), dtype='float32'))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    beta_squared = beta ** 2
    fbeta_score = (1 + beta_squared) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return fbeta_score

## **Define the model**

In [ ]:
model = Sequential()
model.add(Conv2D(64, 5, 2, activation="relu", input_shape=(64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(17, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=[fbeta])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Training model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=45, batch_size=64, callbacks=[early_stopping])

In [ ]:
gc.collect()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

In [ ]:
# Split the data into two parts
test_part1 = sam_sub.iloc[:40669]
test_part2 = sam_sub.iloc[40669:]

# Load and preprocess images from the first directory
test_img_part1 = []
for image_name in tqdm(test_part1['image_name'], miniters=1000):
    arr = cv2.imread(f'../input/planets-dataset/planet/planet/test-jpg/{image_name}.jpg', cv2.IMREAD_UNCHANGED)
    arr = cv2.resize(arr, (64, 64))
    test_img_part1.append(arr)

# Load and preprocess images from the second directory
test_img_part2 = []
for image_name in tqdm(test_part2['image_name'], miniters=1000):
    arr = cv2.imread(f'../input/planets-dataset/test-jpg-additional/test-jpg-additional/{image_name}.jpg', cv2.IMREAD_UNCHANGED)
    arr = cv2.resize(arr, (64, 64))
    test_img_part2.append(arr)


In [ ]:
# Concatenate the two parts of images
test_img = np.concatenate([test_img_part1, test_img_part2], axis=0)

# Normalize the images
test_img = test_img.astype(np.float16) / 255.0

# Predict on the test set
predictions = model.predict(test_img)

# Convert predictions to tags
threshold = 0.0
preds = []

# Iterate through each prediction
for pred in predictions:
    # Get the indices where the prediction is above the threshold
    indices = [i for i, p in enumerate(pred) if p > threshold]

    # Map the indices to their corresponding tags using label_maps
    tags = [label_maps.iloc[i]['tags'] for i in indices]

    # Join the tags into a single string
    preds.append(' '.join(tags))

In [ ]:
# Create submission file
submission = pd.DataFrame({'image_name': sam_sub['image_name'], 'tags': preds})
submission.to_csv('submission.csv', index=False)
